In [1]:
import torch
import torch as th
import numpy as np
from utils import load_data
import torch.nn.functional as F

In [2]:
path = "E:/Abhinab/HGAT/code/HGAT/model/data/agnews/"
dataset = 'agnews'

In [3]:
adj, features, labels, idx_train_ori, idx_val_ori, idx_test_ori, idx_map = load_data(path = path, dataset = dataset)

423it [00:00, 4106.43it/s]

Loading agnews dataset...
Loading text content...
E:/Abhinab/HGAT/code/HGAT/model/data/agnews/
agnews
text


6000it [00:01, 5105.70it/s]
E:\Abhinab\HGAT\code\HGAT\model\code\utils.py:208: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
20it [00:00, 606.07it/s]
105it [00:00, 1039.63it/s]

label matrix shape: torch.Size([6000, 4])
done.
Loading topic content...
E:/Abhinab/HGAT/code/HGAT/model/data/agnews/
agnews
topic
done.
Loading entity content...
E:/Abhinab/HGAT/code/HGAT/model/data/agnews/
agnews
entity


3907it [00:03, 1287.22it/s]


done.
Building graph...
Num of edges: 209847
train, vali, test:  1600 1600 2800


In [5]:
adj[0][0].type()

'torch.sparse.FloatTensor'

In [6]:
[i.shape for i in features]

[torch.Size([6000, 626]), torch.Size([20, 626]), torch.Size([3907, 2785])]

In [7]:
test_adj = torch.Tensor([[1, 2, 3]]).t().repeat(1, 4)
print("A: ")
print(test_adj)
print("method 1: ")
print(test_adj.sum(1).repeat(4, 1).t())
print("method 2: ")
print(th.mm(test_adj, th.ones(4, 4)))


A: 
tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.]])
method 1: 
tensor([[ 4.,  4.,  4.,  4.],
        [ 8.,  8.,  8.,  8.],
        [12., 12., 12., 12.]])
method 2: 
tensor([[ 4.,  4.,  4.,  4.],
        [ 8.,  8.,  8.,  8.],
        [12., 12., 12., 12.]])


In [10]:
torch.where(test_adj > 1, torch.ones_like(test_adj), -9e15*torch.ones_like(test_adj))

tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15, -9.0000e+15],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00]])

In [13]:
F.softmax(test_adj, dim=0)

tensor([[0.0900, 0.0900, 0.0900, 0.0900],
        [0.2447, 0.2447, 0.2447, 0.2447],
        [0.6652, 0.6652, 0.6652, 0.6652]])

In [18]:
torch.stack((th.tensor([1., 2., 3.]),), dim=1)

tensor([[1.],
        [2.],
        [3.]])

In [21]:
# stack test
xt_1 = [th.tensor([1., 2., 3., 4.]), th.tensor([1., 2., 3., 4.]), th.tensor([1., 2., 3., 4.])]
torch.stack(xt_1, dim=1).shape

torch.Size([4, 3])

In [40]:
# stack test 2
# x = torch.cat([x, torch.stack([x[self.idx]] * self.n, dim=0)], dim=2)
x = torch.rand(6000, 3, 50)
x = x.transpose(0, 1)
n = x.size()[0]
# torch.stack([x[0]] * n, dim=0).shape
torch.cat([x, torch.stack([x[0]] * n, dim=0)], dim=2).shape
# len([x[0]] * 3)

torch.Size([3, 6000, 100])

In [47]:
# batch multiplication test
x = torch.rand(3, 6000, 100)
a = torch.rand(100, 1)
# torch.matmul(x, a).shape
F.leaky_relu_(torch.matmul(x, a)).shape

torch.Size([3, 6000, 1])

In [48]:
# batch multiplication test 2
weights = torch.rand(6000, 3, 1)
inputs = torch.rand(6000, 3, 512)
(torch.matmul(weights.transpose(1, 2), inputs).squeeze(1)*3).shape

torch.Size([6000, 512])

In [13]:
#### HGAT model loading:
from models import HGAT

model = HGAT(nfeat_list=[i.shape[1] for i in features],
                    type_attention = True,
                    node_attention = True,
                    nhid=512,
                    nclass=labels.shape[1],
                    dropout=0.5,
                    gamma=0.1,
                    orphan=True,
                 )

In [15]:
model.forward(features, adj)

[tensor([[-2.5089, -2.0249, -1.8353, -1.3351, -2.8303, -1.1876],
         [-2.6877, -1.9871, -2.0454, -1.3810, -2.8728, -1.0281],
         [-2.5822, -1.9165, -1.7132, -1.4914, -2.9023, -1.1487],
         ...,
         [-2.8009, -1.7958, -1.6703, -1.5745, -2.9919, -1.1155],
         [-3.0405, -1.5190, -1.6627, -1.6723, -2.8416, -1.2124],
         [-2.6721, -1.7791, -1.6616, -1.6078, -2.7571, -1.1762]],
        grad_fn=<LogSoftmaxBackward>),
 tensor([[ -1.8333,  -5.3801,  -2.3834,  -0.6439, -11.7857,  -1.5230],
         [ -2.0219,  -3.8736,  -1.6384,  -0.9988,  -7.7968,  -1.2595],
         [ -1.7271,  -3.5366,  -1.8998,  -0.9216,  -7.0563,  -1.4076],
         [ -1.9014,  -3.8579,  -2.0391,  -0.9818,  -7.7670,  -1.1260],
         [ -2.1387,  -3.7785,  -2.0978,  -0.9289,  -7.4695,  -1.0756],
         [ -2.0758,  -3.8195,  -1.9161,  -0.9504,  -6.8066,  -1.1466],
         [ -1.8104,  -3.5515,  -1.8291,  -0.8373,  -6.5199,  -1.5473],
         [ -1.7476,  -3.3436,  -1.6188,  -0.9128,  -6.2517,